# LangGraph: Building Agents with Graph Structures

This Colab addresses **Part A** of the assignment on *Building Effective Agents*. It demonstrates agent design patterns using the **LangGraph API**, focusing on structuring agent interactions as a stateful graph. The implementation is aligned with frameworks introduced in:

- [Building Effective Agents – YouTube walkthrough](https://www.youtube.com/watch?v=aHCDrAbH_go)
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/workflows)
- [DeepLearning.AI Short Course: AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/)

We illustrate key patterns such as **Tool Use** and **Conditional Agent Execution**, and enable LangSmith tracing to observe the full agent lifecycle.

---

## Objective

- Build a LangGraph agent that can reason over a question, determine if a tool is needed, and respond appropriately.
- Showcase tool use (e.g., web search, real-time clock), conditional logic, and looping behavior.
- Enable traceability with LangSmith Studio to inspect the flow between agent nodes.

---

## Key Concepts Demonstrated

- **Graph-based Agent Design** using LangGraph and state transitions
- **Tool Use Pattern** via DuckDuckGo Search and a Custom Time Tool (using Python `datetime`)
- **Conditional Execution** with a decision node that controls whether the agent should call a tool or finalize the response
- **LangSmith Tracing** integration for visual debugging and agent trace walkthroughs
- **Interactive Input Block** for users to ask custom questions and observe how the agent behaves

---

## 1. Setting up LangChain and LangGraph

First, install the necessary libraries.

---

## 2. Setup Environment Variables

Set up your Groq API key. For **LangSmith Tracing** (highly recommended for debugging and observing agent behavior):

1. Go to [langsmith.com](https://www.langsmith.com/) and create an account.
2. Create a new project.
3. Generate an API key.
4. Set the environment variables in your Colab secrets or manually in the notebook.

---

## 3. Define Tools

We use two tools:

- `search`: A web search tool using DuckDuckGo, for answering real-time or unknown questions
- `get_time_in_india`: A custom tool that returns the current time in India using `datetime` and `pytz`

---

## 4. Define the Agent State

The state tracks the conversation history (`messages`). LangGraph manages the state transitions between nodes using a `TypedDict`.

---

## 5. Define the Agent Logic (Nodes)

We define functions that will act as nodes in our graph:

- `call_model`: Invokes the LLM and suggests the next action (respond or tool)
- `should_continue`: Determines whether to call a tool or end the graph
- `call_tool`: Executes the selected tool
- `generate_final_answer`: Produces the final answer when no more tools are needed

---

## 6. Define the Graph

We wire together the nodes and define the conditional edges based on the agent's output.

- **Entry Point**: `agent` (calls the LLM)
- **Nodes**: `agent`, `call_tool`, `generate_final_answer`
- **Edges**: Conditional from `agent`, loop back from `call_tool`, and exit at `generate_final_answer`

---

## 7. Run the Agent

We test with both fixed questions and interactive input. LangSmith tracing (if enabled) captures the execution flow of each step.

---

## 8. Understanding the Patterns

- **Graph Structure**: Built using `StateGraph`. Each node corresponds to a discrete operation in the agent loop.
- **Tool Use**: The LLM determines if a tool like `search` or `get_time_in_india` is needed. Tool output is added to the message state.
- **Conditional Logic**: The agent loop is controlled by evaluating whether tool calls are present.
- **Reflection (Basic)**: Looping from `call_tool → agent` allows the model to revise or complete the task.
- **LangSmith Tracing**: Shows all node transitions, inputs, outputs, and state changes.

---

This Colab satisfies Part A of the assignment by implementing a LangGraph-based agent with tool use, conditional logic, and integrated tracing. It can be extended with additional tools or agent behaviors such as planners or reflection nodes if needed.


In [1]:
!pip install -U langchain-core langchain-community langgraph langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56

In [2]:
!pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.7 MB/s eta 0:00:00


In [4]:
import os
import getpass
from typing import Annotated, List, TypedDict, Tuple # Added Tuple
import json # Added json
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, ToolMessage, SystemMessage # Added SystemMessage
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain.tools import Tool
from langgraph.graph import StateGraph, END # Imported END
from langgraph.graph.message import add_messages
from langchain_core.utils.function_calling import convert_to_openai_tool

In [8]:
# --- Setup Environment Variables ---
# It's better practice to set these in your environment/Colab secrets
try:
    # Check if running in Colab or similar environment with getpass
    import google.colab # Try importing colab specific library
    _api_key = getpass.getpass("Enter your Groq API Key: ")
    _langsmith_key = getpass.getpass("Enter LangSmith API Key (optional, press enter to skip): ")
except (ImportError, ModuleNotFoundError):
     # Fallback for environments without getpass or google.colab
     print("Non-interactive environment detected. Fetching keys from environment variables.")
     _api_key = os.environ.get("GROQ_API_KEY", "")
     _langsmith_key = os.environ.get("LANGCHAIN_API_KEY", "")
     if not _api_key:
          print("WARNING: GROQ_API_KEY environment variable not set.")
     if not _langsmith_key:
          print("INFO: LANGCHAIN_API_KEY environment variable not set. Skipping LangSmith.")


os.environ["GROQ_API_KEY"] = _api_key

# Optional LangSmith Tracing
if _langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = _langsmith_key
    # Try getting project name from env var first
    project_name = os.environ.get("LANGCHAIN_PROJECT")
    if not project_name:
         try:
              # Only prompt if interactive
              import google.colab
              project_name = input("LangSmith Project Name (e.g., LangGraph Groq Demo): ")
         except (ImportError, ModuleNotFoundError):
              project_name = None # Cannot prompt

    if not project_name:
         project_name = "LangGraph Groq Demo" # Default project name if not provided/prompted
    os.environ["LANGCHAIN_PROJECT"] = project_name
    print(f"LangSmith tracing enabled for project: {project_name}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled.")

Enter your Groq API Key: ··········
Enter LangSmith API Key (optional, press enter to skip): ··········
LangSmith Project Name (e.g., LangGraph Groq Demo): pr-memorable-waiter-55
LangSmith tracing enabled for project: pr-memorable-waiter-55


In [9]:
search_tool = DuckDuckGoSearchRun(name="search")
tools = [
    Tool(
        name="search",
        func=search_tool.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this tool should be a single search term string.", # Clarified input type
    )
]
# Convert tools to OpenAI format (required by many models for tool calling)
openai_tools = [convert_to_openai_tool(t) for t in tools]

# Create a Tool Executor map (name -> function) for easy lookup
tools_executor_map = {tool.name: tool.func for tool in tools}


In [10]:
# --- Define the LLM and bind tools ---
# Select a Groq model that supports tool use well
# llama3-70b-8192 is generally recommended
# mixtral-8x7b-32768 is another option
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)

# Bind the tools to the LLM instance.
# This informs the LLM about the available tools and their schemas.
llm_with_tools = llm.bind_tools(openai_tools)

In [11]:
# --- Define the Agent State ---
# TypedDict defines the structure of the state that flows through the graph.
# `add_messages` is a helper function to append messages to the 'messages' list.
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    # Removed intermediate_steps and last_tool_output as they weren't directly used
    # in the node logic. Message history is managed by `add_messages`.

In [12]:
def call_model(state: AgentState):
    """Invokes the LLM with the current state messages and bound tools."""
    print("--- Calling Model ---")
    messages = state["messages"]
    # Invoke the LLM with the messages. The LLM knows about the tools via `bind_tools`.
    response = llm_with_tools.invoke(messages)
    response = chain.invoke({"messages": messages})

    return {"messages": [response]}


In [13]:
def call_tool(state: AgentState):
    """Executes a tool call based on the last AI message."""
    print("--- Calling Tool ---")
    last_message = state["messages"][-1]

    if not isinstance(last_message, AIMessage) or not last_message.tool_calls:
        print("--- No Tool Call Found in Last Message ---")
        return {}

    tool_call = last_message.tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    if tool_name not in tools_executor_map:
        print(f"--- Error: Tool '{tool_name}' not found! ---")
        error_msg = ToolMessage(content=f"Error: Tool '{tool_name}' not found.", tool_call_id=tool_call["id"])
        return {"messages": [error_msg]}

    tool_func = tools_executor_map[tool_name]

    print(f"🛠️ Calling tool: {tool_name} with args: {tool_args}")

    tool_input_arg = None
    if isinstance(tool_args, dict):
        if 'query' in tool_args:
            tool_input_arg = tool_args['query']
        elif '__arg1' in tool_args:
            tool_input_arg = tool_args['__arg1']
        elif tool_args:
            tool_input_arg = list(tool_args.values())[0]
    elif isinstance(tool_args, str):
        tool_input_arg = tool_args

    if tool_input_arg is None:
        print(f"--- Error: Could not determine input string for tool '{tool_name}' from args: {tool_args} ---")
        error_msg = ToolMessage(content=f"Error: Invalid or missing arguments for tool '{tool_name}'. Expected a single search string. Got: {tool_args}", tool_call_id=tool_call["id"])
        return {"messages": [error_msg]}

    if not isinstance(tool_input_arg, str):
        print(f"--- Warning: Converting tool input '{tool_input_arg}' to string for tool '{tool_name}' ---")
        tool_input_arg = str(tool_input_arg)

    try:
        tool_output = tool_func(tool_input_arg)
        print(f"    Tool output type: {type(tool_output)}")
        print(f"    Tool output (truncated): {str(tool_output)[:200]}...\n")
        tool_output_message = ToolMessage(
            content=str(tool_output),
            tool_call_id=tool_call["id"]
        )
        return {"messages": [tool_output_message]}

    except Exception as e:
        print(f"--- Error executing tool {tool_name}: {e} ---")
        error_msg = ToolMessage(content=f"Error executing tool {tool_name}: {e}", tool_call_id=tool_call["id"])
        return {"messages": [error_msg]}

In [14]:
def should_continue(state: AgentState):
    """Determines the next step: call a tool or generate final answer."""
    print("--- Checking Condition: Should Continue? ---")
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        print("Decision: Call Tool")
        return "call_tool"
    else:
        # If no tool call, we assume the agent has provided the final answer
        print("Decision: Generate Final Answer")
        return "generate_final_answer"

def generate_final_answer(state: AgentState):
    """Simply passes the last AI message as the final answer."""
    print("--- Generating Final Answer ---")
    return {"messages": [state["messages"][-1]]}

In [15]:
# --- Define the Graph ---
builder = StateGraph(AgentState)
builder.add_node("agent", call_model)
builder.add_node("call_tool", call_tool)
builder.add_node("generate_final_answer", generate_final_answer) # New node

builder.set_entry_point("agent")

builder.add_conditional_edges(
    "agent",
    should_continue,
    {
        "call_tool": "call_tool",
        "generate_final_answer": "generate_final_answer" # New edge
    }
)

builder.add_edge("call_tool", "agent")
builder.set_finish_point("generate_final_answer") # Set the final node

graph = builder.compile()

In [16]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            "You are a helpful assistant.\n"
            "Only use the tool `search` if you absolutely need real-time data about current events.\n"
            "If the answer is general knowledge (e.g., capital cities, historical facts), respond directly in natural language.\n"
            "Never format tool-use when not using a tool."
        )
    ),
    MessagesPlaceholder(variable_name="messages"),
])


In [17]:
chain = prompt | llm_with_tools


In [18]:
print("\n🤖 Running Agent for Weather Question...\n")
try:
    result = graph.invoke(
        {"messages": [HumanMessage(content="What is the current weather in Milpitas, California?")]},
        config={"recursion_limit": 50},
    )

    final_response_weather = None
    if result and "messages" in result:
        for msg in result["messages"]:
            if isinstance(msg, AIMessage):
                final_response_weather = msg

    if final_response_weather:
        print("\n🎯 Final Answer (Weather):", final_response_weather.content)
    else:
        print("\n❌ Could not extract final AI message for weather.")

except Exception as e:
    print(f"\n--- Error during weather agent run: {e} ---")
    import traceback
    traceback.print_exc()



🤖 Running Agent for Weather Question...

--- Calling Model ---
--- Checking Condition: Should Continue? ---
Decision: Call Tool
--- Calling Tool ---
🛠️ Calling tool: search with args: {'__arg1': 'current weather in Milpitas, California'}
    Tool output type: <class 'str'>
    Tool output (truncated): Current conditions at San Jose, San Jose International Airport (KSJC) ... Milpitas CA 37.42°N 121.92°W (Elev. 20 ft) Last Update: 2:21 am PDT Apr 28, 2025. ... Severe Weather ; Current Outlook Maps ; ...

--- Calling Model ---
--- Checking Condition: Should Continue? ---
Decision: Generate Final Answer
--- Generating Final Answer ---

🎯 Final Answer (Weather): The current weather in Milpitas, California is overcast with a temperature of 57°F (feeling like 56°F), visibility of 9 miles, and a UV index of 1.1.


In [19]:
print("\n🤖 Running Agent for Direct Question...\n")
try:
    result_direct = graph.invoke(
        {"messages": [HumanMessage(content="What is the capital of France?")]},
        config={"recursion_limit": 50},
    )

    final_response_direct = None
    if result_direct and "messages" in result_direct:
        for msg in result_direct["messages"]:
            if isinstance(msg, AIMessage):
                final_response_direct = msg

    if final_response_direct:
        print("\n🎯 Final Answer (Capital):", final_response_direct.content)
    else:
        print("\n❌ Could not extract final AI message for capital.")

except Exception as e:
    print(f"\n--- Error during direct question agent run: {e} ---")
    import traceback
    traceback.print_exc()



🤖 Running Agent for Direct Question...

--- Calling Model ---
--- Checking Condition: Should Continue? ---
Decision: Generate Final Answer
--- Generating Final Answer ---

🎯 Final Answer (Capital): The capital of France is Paris.


In [20]:
# --- Interactive Question Block ---
try:
    import google.colab
    from IPython.display import display, HTML
    display(HTML("<h3>🔍 Ask Anything (Agent will decide whether to use tools)</h3>"))
except:
    pass

from datetime import datetime

custom_question = input("💬 Enter your question: ")

print(f"\n🤖 Running Agent...\nQ: {custom_question}\n")
try:
    result_dynamic = graph.invoke(
        {"messages": [HumanMessage(content=custom_question)]},
        config={"recursion_limit": 50},
    )

    final_response_dynamic = None
    if result_dynamic and "messages" in result_dynamic:
        for msg in result_dynamic["messages"]:
            if isinstance(msg, AIMessage):
                final_response_dynamic = msg

    if final_response_dynamic:
        print(f"\n🎯 Final Answer: {final_response_dynamic.content}")
    else:
        print("\n❌ Could not extract final AI message.")

except Exception as e:
    print(f"\n--- Error during custom agent run: {e} ---")
    import traceback
    traceback.print_exc()


💬 Enter your question: What are the recents events happening around the world that effects climate change

🤖 Running Agent...
Q: What are the recents events happening around the world that effects climate change

--- Calling Model ---
--- Checking Condition: Should Continue? ---
Decision: Call Tool
--- Calling Tool ---
🛠️ Calling tool: search with args: {'__arg1': 'climate change recent events'}
    Tool output type: <class 'str'>
    Tool output (truncated): This alarming trend is attributed to human-induced climate change, compounded by natural phenomena such as El Niño. ... NOAA officials noted in October that the current event has surpassed previous re...

--- Calling Model ---
--- Checking Condition: Should Continue? ---
Decision: Generate Final Answer
--- Generating Final Answer ---

🎯 Final Answer: Based on the tool call result, here's a summary of recent climate change events:

* The current El Niño event has broken records, surpassing previous records by over 11%, and is still

## LangGraph Agent Demonstration – Tool Use Pattern with Conditional Logic

This notebook demonstrates the **Tool Use** and **Conditional Agent Execution** patterns as described in "Building Effective Agents". The implementation uses the **LangGraph API** along with **LangChain tools** to build an interactive, traceable agent that can respond to user queries and decide whether to use tools based on the question.

### Key Concepts Implemented

- **LangGraph API**: Used to build a graph-based agent where nodes represent specific functions such as LLM invocation, tool calls, and final answer generation.
- **Tool Use Pattern**: The agent is capable of calling external tools (e.g., web search or current time in India) based on the input query.
- **State Management**: Agent state is passed between nodes, maintaining the history of messages and tool interactions.
- **Conditional Logic**: Implemented using `should_continue()` to determine whether the agent should call a tool or generate a final response.
- **Custom Tool Integration**: In addition to DuckDuckGo search, a custom tool was added using `datetime` and `pytz` to return the current time in India.
- **Interactive Block**: Users can input any question in a prompt, and the agent dynamically decides how to respond.
- **LangSmith Integration**: LangSmith tracing is enabled to visually track the execution of each node in the agent graph, including message flow and tool outputs.

### Execution Flow

1. **User Input**: A question is passed to the graph.
2. **Model Node** (`call_model`): The agent reasons over the question and decides whether it needs to use a tool.
3. **Conditional Edge**: Based on the model output, the flow proceeds to either `call_tool` or `generate_final_answer`.
4. **Tool Node** (`call_tool`): If a tool is required, the agent calls the appropriate function and integrates the response.
5. **Loopback**: The agent re-evaluates the updated message state to decide if further steps are needed.
6. **Final Node**: Once the agent produces a complete response, the graph ends at the `generate_final_answer` node.

### Example Queries Tested

- "What is the capital of France?"
- "What is the current weather in Milpitas, California?"
- "What is the current time in India?"

For questions requiring real-time or dynamic data, the agent utilizes the tools. For general knowledge questions, it responds directly without calling any tools.

### Next Steps

- Upload this Colab notebook to GitHub as part of the assignment submission.
- Record a walkthrough video showing:
  - How the agent executes queries
  - How conditional tool use works
  - LangSmith trace visualization for a complete execution

This notebook satisfies the requirements for Part A of the assignment by demonstrating effective agent behavior using the LangGraph API and tool integrations.
